- 本章介绍高级RDD操作，重点介绍键值RDD
- 使用自定义分区也是我们使用RDD的原因之一，借此可以精确控制数据在集群上的分布
- 同时也会讨论RDD连接

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python").getOrCreate()

In [7]:
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")

In [8]:
words = spark.sparkContext.parallelize(myCollection, 2)

<h4>Key-Value基础（Key-Value RDD）</h4>

- 这种方法都有形如\<some-operation>ByKey的API名称
- 这意味着只能以PairRDD类型执行此操作
- 最简单的方法就是将当前的RDD映射到基本的key-value结构
- 也就是说RDD的每个记录都有两个值

In [5]:
words.map(lambda word: (word.lower(), 1)).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 0.0 failed 1 times, most recent failure: Lost task 1.0 in stage 0.0 (TID 1) (26.26.26.1 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.

<h4>keyBy</h4>

- 不仅可以使用map创建元组对
- 还可以使用keyBy函数，根据value创建key的函数
- 比如以下例子中，将第一个字母作为key，然后Spark将单词记录保存为RDD的value

In [ ]:
keyword = words.keyBy(lambda word: word.lower()[0])

<h4>对值进行映射</h4>

- 在有一组键值对后，Spark将假设第一个元素是key，第二个元素是value
- 这时我们就可以只对值进行映射
- 以下例子就是将值变为upper

In [ ]:
keyword.mapValues(lambda word: word.upper()).collect()

<h4>flatMapValues</h4>

- 也可以在行上进行flatMap操作来扩展行数
- 每行表示一个字符
- 下面的例子将输出每个字符，将单词转换为字符串数组

In [ ]:
keyword.flatMapValues(lambda word: word.upper()).collect()

<h4>提取key和value</h4>

In [ ]:
keyword.keys().collect()
keyword.values().collect()

<h4>lookup</h4>

- 查找某个key对应的value
- Spark并不强制规定键不能重复
- 以下例子中，如果查找s，会得到两个，即"Spark"和"Simple"

In [ ]:
keyword.lookup("s")

<h4>sampleByKey</h4>

In [12]:
import random

In [15]:
distinctChars = words.flatMap(lambda word: list(word.lower())).distinct.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 4) (26.26.26.1 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:182)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:107)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:119)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:145)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:373)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:337)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.

In [ ]:
sampleMap = dict(map(lambda c: (c, random.random()), distinctChars))

In [ ]:
words.map(lambda word: (word.lower()[0], word)).sampleByKey(True, sampleMap, 6).collect()

<h4>聚合操作</h4>

- 可以在纯RDD或PairRDD执行聚合操作

In [16]:
chars = words.flatMap(lambda word: word.lower())
KVcharacters = chars.map(lambda letter: (letter, 1))

def maxFunc(left, right):
    return max(left, right)

def addFunc(left, right):
    return left + right

In [27]:
nums = spark.sparkContext.parallelize(range(1,32), 5)

<h4>countByKey</h4>

- 计算每个key对应的数据项的数量，并将结果写到本地Map中

In [ ]:
KVcharacters.countByKey()

<h4>了解聚合操作的实现</h4>

- 比较两个基本方法：groupBy和reduce
- 我们手里的pairRDD为(字母，1)，下面例子统计各个字母的总计数

In [23]:
from functools import reduce
reduce(lambda x, y: x + y, [1,2,3])

6

In [24]:
KVcharacters.groupByKey().map(lambda row: (row[0], reduce(addFunc, row[1])))

PythonRDD[9] at RDD at PythonRDD.scala:53

- 但在大多数情况下，这是错误的方法
- 问题在于每个执行器在执行函数之前，必须在内存中保存一个key对应的所有value
- 如果有严重的key负载倾斜现象，即key对应着太多的value而导致超载问题
- 进而出现OutOfMemoryErrors错误
- 处理大规模数据时会出现这个问题
- 只有在每个key的value数量都差不多，且能够被执行器的内存容纳的情况下，才可以使用groupByKey方法

<h4>reduceByKey</h4>

1. 首先通过flatMap将value的值，单词变为字母
2. 然后通过map方法，将字母映射为(字母，1)
3. 然后执行reduceByKey配以求和函数以将结果存储到数组
4. 以下返回每个字母的计数

In [ ]:
KVcharacters.reduceByKey(addFunc).collect()

<h4>其他聚合方法</h4>

<h4>aggregate</h4>

- 此函数需要一个null值和一个起始值
- 并需要制定两个不同的函数，第一个函数执行分区内聚合，第二个执行分区间聚合
- 起始值在两个聚合级别使用

In [ ]:
nums = spark.sparkContext.parallelize(range(1,32), 5)

In [ ]:
nums.aggregate(0, maxFunc, addFunc)

- aggregate有一些性能问题
- 由于它在驱动器上执行最终聚合，如果执行器的结果太大，则会导致驱动器出现OutOfMemory错误并让程序崩溃
- 还有一个方法是treeAggregate，它基于不同实现方法可以得到与aggregate相同的结果
- nums.treeAggregat(0, maxFunc, addFunc, depth=3)

<h4>aggregateByKey</h4>

- 此函数与aggregate基本相同，但是基于key聚合而非基于分区聚合
- 起始值和函数的属性配置也都相同

<h4>combineByKey</h4>

- 总共三个合并函数
    1. 对某个key进行操作
    2. 对value进行合并
    3. 合并各个合并器的输出结果

In [28]:
def valToCombiner(value):
    return [value]

def mergeValuesFunc(vals, valToAppend):
    vals.append(valToAppend)
    return vals

def mergeCombinerFunc(vals1, vals2):
    return vals1 + vals2

In [ ]:
KVcharacters.combineByKey(vars=valToCombiner, mergeValue=mergeValuesFunc, mergeCombiners=mergeCombinerFunc)

<h4>foldByKey</h4>

In [ ]:
KVcharacters.foldByKey(0, addFunc).collect()

<h4>CoGroups</h4>

- 在Scala中允许将三个key-value RDD一起分组
- 在Python中允许将两个key-value RDD一起分组，基于key连接value

In [29]:
import random

In [30]:
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
words = spark.sparkContext.parallelize(myCollection, 2)

In [ ]:
distinctChars = words.flatMap(lambda word: word.lower()).distinct()
charRDD = distinctChars.map(lambda c: (c, random.random()))
charRDD2 = distinctChars.map(lambda c: (c, random.random()))
charRDD.cogroup(charRDD2).take(5)

<h4>连接操作</h4>

- RDD连接操作API需要的参数有
- 执行连接操作的两个RDD，输出分区数，自定义分区函数

<h4>内连接</h4>

In [ ]:
keyedChars = distinctChars.map(lambda c: (c, random.random))
outputPartitions = 10
KVcharacters.join(keyedChars, outputPartitions).count()

- 其他连接类型
    - 全外连接（fullOuterJoin）
    - 左外连接（leftOuterJoin）
    - 右外连接（rightOuterJoin）
    - 笛卡尔连接（cartesian，该连接非常危险，它不接受连接key，并且可能会有大量的输出）

<h4>zip</h4>

- zip并不是一个连接操作，但它将两个RDD组合在一起
- zip把两个RDD怨怒对应的匹配在一起，要求两个RDD的元素个数相同
- 同时也要求两个RDD分区数相同，结果会产生一个PairRDD

In [31]:
numRange = spark.sparkContext(range(10), 2)

TypeError: 'SparkContext' object is not callable

In [ ]:
words.zip(numRange).collect()

<h4>控制分区</h4>

- 使用RDD，你可以控制数据在整个集群上的物理分布
- 其中一些方法与结构化API基本相同
- 但是最关键的区别在于，RDD可以指定一个数据分区函数

<h4>coalesce</h4>

- coalesce有效折叠（collapse）同一工作节点上的分区，以便在重新分区时避免数据洗牌（shuffle）
- 例如，存储words变量的RDD当前有两个分区，可以用coalesce折叠为一个分区 ，从而避免了数据shuffle

In [ ]:
words = spark.sparkContext.parallelize(myCollection, 2)

In [ ]:
words.coalesce(1).getNumPartitions()

<h4>repartition</h4>

- 对数据进行重新分区，跨节点的分区会执行shuffle操作
- 对于map操作和filter操作，增加分区可以提高并行度

In [33]:
# 变为10个分区
words.repartition(10)

MapPartitionsRDD[16] at coalesce at NativeMethodAccessorImpl.java:0

<h4>自定义分区</h4>

- 自定义分区的作用时控制集群上数据的分布并避免shuffle操作
- 由于结构化API不支持自定义数据分区
- 我们首先应该从结构化API定义的数据降级为RDD
- 应用自定义分区程序
- 然后再将RDD转换为DataFrame或Dataset

- 以下例子将分区数减为10个

In [35]:
df = spark.read.option("header", "true").option("inferSchema", "true").csv("./data/retail-data/all/online-retail-dataset.csv")

In [40]:
df.rdd.getNumPartitions()

11

In [43]:
rdd = df.coalesce(10).rdd
rdd.getNumPartitions()

10

<h4>Spark有两个内置的分区器，可以在RDD API中调用</h4>

- 用于离散值划分的HashPartitioner（基于哈希的分区）
- 用于连续纸划分的RangePartitioner（根据数据范围分区）
- 结构化API和RDD均可调用


- 有时由于数据量很大，并且存在严重的数据倾斜
- 比如某些key对应的value值，比其他key多很多
- 我们就需要通过底层的分区方法，对这个key进行更细粒度的分解
- 以防止OutOfMemoryError报错

- 比如下面这个例子将两个键17850和12583划入第0分区
- 剩余键随机放入第1，2分区

In [44]:
def partitionFunc(key):
    import random
    if key == 17850 or key == 12583:
        return 0
    else:
        return random.randint(1,2)

In [51]:
df.show(3)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 3 rows



- 取CustomerID作为键

In [50]:
keyedRDD = rdd.keyBy(lambda row: row[6])

2

In [ ]:
keyedRDD.partitionBy(3, partitionFunc)

In [ ]:
keyedRDD.map(lambda x: x[0]).glom().map(lambda x: len(set(x))).take (5)